### Ingest results.json file

In [ ]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

dbutils.widgets.text("p_file_date", "2021-03-28")
v_file_date = dbutils.widgets.get("p_file_date")

In [ ]:
from formula1.formula1_constants import *
from formula1.formula1_utils import *

##### Step 1 - Read the JSON file using the spark dataframe reader API

In [ ]:
from pyspark.sql.types import (
    StructType,
    StructField,
    IntegerType,
    StringType,
    FloatType,
)
from pyspark.sql.functions import lit

results_schema = StructType(
    fields=[
        StructField("resultId", IntegerType(), False),
        StructField("raceId", IntegerType(), True),
        StructField("driverId", IntegerType(), True),
        StructField("constructorId", IntegerType(), True),
        StructField("number", IntegerType(), True),
        StructField("grid", IntegerType(), True),
        StructField("position", IntegerType(), True),
        StructField("positionText", StringType(), True),
        StructField("positionOrder", IntegerType(), True),
        StructField("points", FloatType(), True),
        StructField("laps", IntegerType(), True),
        StructField("time", StringType(), True),
        StructField("milliseconds", IntegerType(), True),
        StructField("fastestLap", IntegerType(), True),
        StructField("rank", IntegerType(), True),
        StructField("fastestLapTime", StringType(), True),
        StructField("fastestLapSpeed", FloatType(), True),
        StructField("statusId", StringType(), True),
    ]
)

In [ ]:
results_df = spark.read.schema(results_schema).json(
    f"{landing_folder_path}/{v_file_date}/results.json"
)

##### Step 2 - Add ingestion meta data

In [ ]:
results_with_metadata_df = (
    add_ingestion_date(results_df)
    .withColumn("data_source", lit(v_data_source))
    .withColumn("file_date", lit(v_file_date))
)

##### Step 3 - Write to output to bronze container

In [ ]:
results_with_metadata_df.write.mode("overwrite").partitionBy("raceId").format(
    "delta"
).saveAsTable("f1_bronze.results")